In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in data from crew csv
crew = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.crew.tsv', sep='\t')
ratings = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.ratings.tsv', sep='\t')
titles = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.basics.tsv', sep='\t')


/var/folders/b9/1w_dn_kj5zgffq25ztcmwj5m0000gn/T/ipykernel_1305/1931529237.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.basics.tsv', sep='\t')


In [3]:
titles.replace('\\N', np.nan, inplace=True)
titles['startYear'] = titles['startYear'].astype(float)

In [4]:
titles.tail()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
9273127,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010.0,NaN,NaN,"Action,Drama,Family"
9273128,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010.0,NaN,NaN,"Action,Drama,Family"
9273129,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010.0,NaN,NaN,"Action,Drama,Family"
9273130,tt9916856,short,The Wind,The Wind,0,2015.0,NaN,27,Short
9273131,tt9916880,tvEpisode,Horrid Henry Knows It All,Horrid Henry Knows It All,0,2014.0,NaN,10,"Adventure,Animation,Comedy"


In [5]:
between_1950_2020_titles = titles[(titles['startYear']>=1950) & (titles['startYear']<=2020) & (titles['titleType'] != 'tvEpisode')]['tconst']

In [6]:
#Heuristic: Let's filter out movies with less than 10000 votes

high_votes = ratings[ratings['numVotes'] > 10000]

In [7]:
votes_and_titles = pd.merge(between_1950_2020_titles, high_votes, on='tconst')

In [9]:
#Turn the directors column into a list
crew['directors'] = crew['directors'].str.split(',')

In [10]:
# explode directors column
directors = crew[['tconst', 'directors']].explode('directors')


In [11]:
# Merge the directors with the titles
directors_and_titles = pd.merge(directors, votes_and_titles, on='tconst')

In [12]:
from sklearn.preprocessing import normalize

In [21]:
#Create 3 columns with votes, normalized votes, and rankings
director_votes = directors_and_titles.groupby('directors')['numVotes'].sum()
normalized_director_votes = normalize(director_votes.values.reshape(1, -1))
ranked_director_votes = director_votes.rank(ascending=False)
final_director_rankings = pd.DataFrame({'director':director_votes.index, 'rank':ranked_director_votes.values, 'total_votes':director_votes.values, 'normalized_votes': normalized_director_votes[0]})
final_director_rankings.sort_values('rank', inplace=True)
#final_director_rankings.to_csv('../data/interim/director_rankings.csv', index=False)

In [15]:
names = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/name.basics.tsv', sep='\t')

In [22]:
# merge names with director rankings
final_director_rankings = pd.merge(final_director_rankings, names, left_on='director', right_on='nconst', how='left')
final_director_rankings.drop(columns=['nconst', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)

In [23]:
final_director_rankings.head()

,director,rank,total_votes,normalized_votes,primaryName
0,nm0634240,1.0,13965885,0.191970,Christopher Nolan
1,nm0000229,2.0,13326650,0.183184,Steven Spielberg
2,nm0000233,3.0,11033530,0.151663,Quentin Tarantino
3,nm0000217,4.0,9540815,0.131145,Martin Scorsese
4,nm0000399,5.0,9033019,0.124165,David Fincher


Now Repeat with the Writers

In [24]:
crew['writers'] = crew['writers'].str.split(',')
writers = crew[['tconst', 'writers']].explode('writers')

In [25]:
writers_and_titles = pd.merge(writers, votes_and_titles, on='tconst')

In [26]:

writer_votes = writers_and_titles.groupby('writers')['numVotes'].sum()
normalized_writer_votes = normalize(writer_votes.values.reshape(1, -1))
ranked_writer_votes = writer_votes.rank(ascending=False)
final_writer_rankings = pd.DataFrame({'writer':writer_votes.index, 'rank':ranked_writer_votes.values, 'total_votes':writer_votes.values, 'normalized_votes': normalized_writer_votes[0]})
#final_writer_rankings.to_csv('../data/interim/writer_rankings.csv', index=False)
final_writer_rankings.sort_values('rank', inplace=True)


In [27]:
# merge names with director rankings
final_writer_rankings = pd.merge(final_writer_rankings, names, left_on='writer', right_on='nconst', how='left')
final_writer_rankings.drop(columns=['nconst', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)

In [28]:
final_writer_rankings.head()

,writer,rank,total_votes,normalized_votes,primaryName
0,nm0498278,1.0,19928451,0.249145,Stan Lee
1,nm0456158,2.0,18396525,0.229993,Jack Kirby
2,nm0634240,3.0,14449606,0.180648,Christopher Nolan
3,nm0000184,4.0,12630074,0.157901,George Lucas
4,nm0004170,5.0,11501813,0.143795,Bob Kane


In [29]:
crew

,tconst,directors,writers
0,tt0000001,[nm0005690],[\N]
1,tt0000002,[nm0721526],[\N]
2,tt0000003,[nm0721526],[\N]
3,tt0000004,[nm0721526],[\N]
4,tt0000005,[nm0005690],[\N]
...,...,...,...
9275709,tt9916848,"[nm5519375, nm5519454]","[nm6182221, nm1628284, nm2921377]"
9275710,tt9916850,"[nm5519454, nm5519375]","[nm6182221, nm1628284, nm2921377]"
9275711,tt9916852,"[nm5519454, nm5519375]","[nm6182221, nm1628284, nm2921377]"
9275712,tt9916856,[nm10538645],[nm6951431]


Now for writers

In [20]:
principals = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.principals.tsv', sep='\t')

In [21]:
# get unique categories from the principals
principals['category'].unique()

array(['self', 'director', 'cinematographer', 'composer', 'producer',
       'editor', 'actor', 'actress', 'writer', 'production_designer',
       'archive_footage', 'archive_sound'], dtype=object)

In [9]:
# only keep actor categories
principals = principals[principals['category'].isin(['actor', 'actress'])]

In [10]:
actors_and_titles = pd.merge(principals, votes_and_titles, on='tconst')

In [13]:
actors_votes = actors_and_titles.groupby('nconst')['numVotes'].sum()
normalized_actor_votes = normalize(actors_votes.values.reshape(1, -1))
ranked_actor_votes = actors_votes.rank(ascending=False)
final_actor_rankings = pd.DataFrame({'actor':actors_votes.index, 'rank':ranked_actor_votes.values, 'total_votes':actors_votes.values, 'normalized_votes': normalized_actor_votes[0]})
#final_writer_rankings.to_csv('../data/interim/writer_rankings.csv', index=False)
final_actor_rankings.sort_values('rank', inplace=True)


In [16]:
# merge names with director rankings
final_actor_rankings = pd.merge(final_actor_rankings, names, left_on='actor', right_on='nconst', how='left')
final_actor_rankings.drop(columns=['nconst', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)

In [22]:
final_actor_rankings.head(20)

,actor,rank,total_votes,normalized_votes,primaryName
0,nm0000093,1.0,16486392,0.163386,Brad Pitt
1,nm0000138,2.0,15165809,0.150298,Leonardo DiCaprio
2,nm0000158,3.0,15163169,0.150272,Tom Hanks
3,nm0000134,4.0,14179951,0.140528,Robert De Niro
4,nm0000168,5.0,12947706,0.128316,Samuel L. Jackson
5,nm0000288,6.0,12767307,0.126528,Christian Bale
6,nm0000375,7.0,12361664,0.122508,Robert Downey Jr.
7,nm0000246,8.0,12158498,0.120495,Bruce Willis
8,nm0000354,9.0,11593490,0.114895,Matt Damon
9,nm0000151,10.0,11344567,0.112429,Morgan Freeman
